# Instructions

1. First create your python environment using Anaconda or Virtualenv with `python>=3.10`.
2. Install prerequisites: `pip install torch numpy matplotlib PyYAML`.
3. Install auxilary package "comblearn" on the project root: `python setup.py develop`.
4. Run "Initialization" part of the notebook.
5. For the first experiment (Learning Coverage Function) run Part 1 of the notebook.
6. For the second experiment (Maximizing Social Welfare) run Part 2 of the notebook.
7. For each experiment use the trial config to generate desired output.

## Initialization

In [7]:
import torch
import torch.nn as nn
import logging

from comblearn.env import CombinatorialAuction

logging.basicConfig(level=20)

In [8]:
vf_configs = ['edsf', 'dsf', 'dpst', 'strf']

## 1. Learning Coverage and Cut Functions

### Trial Config

In [9]:
# Try different configs to get the desired ouputs

config = {
    'prob': 'low', # ['low', 'mid', 'high'],
    'function': 'edsf', # ['dsf', 'edsf', 'egdsf', 'dpst', 'strf']
    'true': 'coverage' # ['coverage', 'cut']
}

In [10]:
import yaml 

if config['true'] == 'coverage':
    with open("config_coverage.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)
else:
    with open("config_cut.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader) 

In [14]:
auction = CombinatorialAuction(cfg['auction'])

In [15]:
if config['prob'] == 'low':
    X, y = auction.data_handler.R['0']
    vfa = auction.bidders[0].vf
elif config['prob'] == 'mid':
    X, y = auction.data_handler.R['1']
    vfa = auction.bidders[1].vf
else:
    X, y = auction.data_handler.R['2']
    vfa = auction.bidders[2].vf
items = auction.items

In [16]:
n = X.shape[0]
tr = int(80/100 * n)
X_train = X[0: tr]
y_train = y[0: tr] 
X_test = X[tr:]
y_test = y[tr:] 

In [18]:
from comblearn.data import ExtendedDSFValueFunction, ExtendedGeneralDSFValueFunction
from comblearn.data import DSFValueFunction, DeepSets, SetTransformer

if config['function'] == 'edsf':
    vf = ExtendedDSFValueFunction(items, 60, [64, 64, 64, 64], 95).to("cuda")
elif config['function'] == 'egdsf':
    vf = ExtendedGeneralDSFValueFunction(items, 60, [64, 64, 64, 64, 10], 95).to("cuda")
elif config['function'] == 'dpst':
    vf = DeepSets(items, [64, 64, 64], [64, 64, 64]).to('cuda')
elif config['function'] == 'strf':
    vf = SetTransformer(items, 1, 1, dim_hidden=128).to("cuda")
else:
    vf = DSFValueFunction(items, 60, [64, 64, 64], 95).to("cuda")

In [ ]:
from torch.optim import Adam

optimizer = Adam(vf.parameters(), lr=0.01)
criterion = nn.L1Loss()

losses = []

for _ in range(4000):
    optimizer.zero_grad()
    yp = vf(X_train)
    loss = criterion(yp, y_train)
    loss.backward(retain_graph=True)
    losses.append(loss.item())
    optimizer.step()
    with torch.no_grad():
        vf.relu()
    print(loss)

In [ ]:
# To chek the loss on test set

yp = vf(X_test)
criterion(yp, y_test).cpu().detach().numpy()

In [ ]:
# To plot the loss throughout the iterations

import matplotlib.pyplot as plt
import numpy as np

x = np.arange(len(losses))
plt.figure(figsize=(25, 20))
plt.title('Loss Function', fontdict={'size': 50})
plt.xlabel('iteration number', fontdict={'size': 42})
plt.ylabel('loss', fontdict={'size': 42})
plt.yticks(np.arange(min(losses), max(losses), (max(losses) - min(losses))//10 + 1), fontsize=30)
plt.xticks(fontsize=30)
plt.plot(x, losses, label='coverage_function', color='black')
plt.savefig("loss.png")
plt.show()

In [ ]:
# To plot the value of some of the training data after training and compare them to the real values

import matplotlib.pyplot as plt

x = range(X_train[0: 50].shape[0])
y1 = vf(X_train[0: 50]).squeeze().tolist()
y2 = y_train[0: 50].squeeze().tolist()

plt.figure(figsize=(25, 15))

plt.title('True vs Predicted Value', fontdict={'size': 40})
plt.xlabel('Train Samples', fontdict={'size': 30})
plt.ylabel('Value of Train Samples', fontdict={'size': 30})
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(x, y1, 'o--', label = "predict")
plt.plot(x, y2, '^--', label = "truth")
plt.legend(loc='lower left', fontsize=27)
plt.savefig("train.png")
plt.show()

In [ ]:
# To plot the value of some of the test data after training and compare them to the real values

import matplotlib.pyplot as plt

x = range(X_test[0: 50].shape[0])
y1 = vf(X_test[0: 50]).squeeze().tolist()
y2 = y_test[0: 50].tolist()
print(vf(X_test[0: 50]).size())

plt.figure(figsize=(25, 15))

plt.title('True vs Predicted Value', fontdict={'size': 40})
plt.xlabel('Test Samples', fontdict={'size': 30})
plt.ylabel('Value of Test Samples', fontdict={'size': 30})
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(x, y1, 'o--', label = "predict")
plt.plot(x, y2, '^--', label = "truth")
plt.legend(loc='lower left', fontsize=27)
plt.savefig("test.png")
plt.show()

### Multiple Running

In [5]:
from comblearn.data import ExtendedDSFValueFunction, DSFValueFunction, DeepSets, SetTransformer
from torch.optim import Adam
from torch import nn

def trial(no, tr_losses):
    print(f"Trial {no}:")

    auction = CombinatorialAuction(cfg['auction'])
    if config['prob'] == 'low':
        X, y = auction.data_handler.R['0']
    elif config['prob'] == 'mid':
        X, y = auction.data_handler.R['1']
    else:
        X, y = auction.data_handler.R['2']
    items = auction.items

    n = X.shape[0]
    tr = int(80/100 * n)
    X_train = X[0: tr]
    y_train = y[0: tr] 
    X_test = X[tr:]
    y_test = y[tr:]

    for vfc in vf_configs:
        print(f'Config {vfc}:')
        if vfc == 'edsf':
            vf = ExtendedDSFValueFunction(items, 60, [64, 64, 64, 64], 95).to("cuda")
        elif vfc == 'dpst':
            vf = DeepSets(items, [512, 512, 512, 512], [512, 512, 512, 512]).to('cuda')
        elif vfc == 'dsf':
            vf = DSFValueFunction(items, 60, [64, 64, 64], 95).to("cuda")
        elif vfc == 'strf':
            vf = SetTransformer(items, num_outputs=1, dim_output=1, dim_hidden=64).to("cuda")

        optimizer = Adam(vf.parameters(), lr=0.01)
        criterion = nn.L1Loss()

        losses = []

        for _ in range(5000):
            optimizer.zero_grad()
            yp = vf(X_train)
            loss = criterion(yp, y_train)
            loss.backward(retain_graph=True)
            losses.append(loss.item())
            optimizer.step()
            with torch.no_grad():
                vf.relu()
            if _ % 1000 == 0:
                print(loss)
        train_loss = loss

        yp = vf(X_test)
        test_loss = criterion(yp, y_test)

        print(f'Train, Test loss: {[train_loss.item(), test_loss.item()]}')
        tr_losses[vfc].append([train_loss.item(), test_loss.item()])
    

In [6]:
trial_losses = {'edsf': [], 'dsf': [], 'dpst': [], 'strf': []}

## 2. Maximizing Social Welfare

### Config

In [3]:
config = {
    'function': 'edsf' # ['dsf', 'edsf', 'vnn', 'comp', 'comp_dsf']
}

In [4]:
import yaml 

if config['function'] == 'edsf':
    with open("config_edsf_social_welfare.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)
elif config['function'] == 'vnn':
    with open("config_vnn_social_welfare.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)
elif config['function'] == 'comp':
    with open("config_comp_social_welfare.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)
elif config['function'] == 'comp_dsf':
    with open("config_dsf_comp_social_welfare.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)
else: 
    with open("config_dsf_social_welfare.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)

In [5]:
device = "cuda"

def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [ ]:
auction = CombinatorialAuction(cfg['auction'])

### Randomized Greedy

In [ ]:
allocation, social_welfare = auction.run()

In [ ]:
social_welfare, allocation

### Gradient Ascent

In [ ]:
auction.allocation_config['scheme'] = 'GradientAscent'
auction.allocation_config['optimizer'] = 'comblearn.optim.GradientAscentOptimizer'  # Change this to batch if you want to perform a batch optimization
auction.allocation_config['learning-rate'] = 0.001
auction.allocation_config['batch-size'] = 100
auction.allocation_config['eps'] = 0.00

In [ ]:
# allocation, social_welfare, allocation1, social_welfare1 = auction.run()
allocation, social_welfare = auction.run()

In [ ]:
allocation, social_welfare #, allocation1, social_welfare1

In [ ]:
print(f'optimal social welfare for auction with coverage value funciton is: {social_welfare}')
print(f'final allocation for auction with coverage value functions is: {allocation}')

### Compare with Randomized Greedy

In [ ]:
from time import time

output = []
for i in range(3):
    auction = CombinatorialAuction(cfg['auction'])
    optimal_social_welfare = auction.data_handler.opt_sw

    auction.allocation_config['scheme'] = 'RandGreedy'
    auction.allocation_config['optimizer'] = 'comblearn.optim.RandGreedyOptimizer'  # Change this to batch if you want to perform a batch optimization
    auction.allocation_config['delta'] = 0.01
    auction.allocation_config['batch-size'] = 100
    auction.allocation_config['sample_rate'] = 5
    s_rg = time()
    _, rg_sw = auction.run()
    e_rg = time()
    auction.allocation_config['scheme'] = 'GradientAscent'
    auction.allocation_config['optimizer'] = 'comblearn.optim.GradientAscentOptimizer'  # Change this to batch if you want to perform a batch optimization
    auction.allocation_config['learning-rate'] = 0.001
    auction.allocation_config['batch-size'] = 100
    auction.allocation_config['eps'] = 0.00
    s_ga = time()
    _, ga_sw = auction.run()
    e_ga = time()

    output.append((rg_sw, ga_sw, e_rg - s_rg, e_ga - s_ga, optimal_social_welfare))

    print('A:', (rg_sw, ga_sw, e_rg - s_rg, e_ga - s_ga, optimal_social_welfare))

In [11]:
os = [[a.item(), b.item(), c, d] for a, b, c, d, _ in output]

In [ ]:
import numpy as np

a = np.array(os)
a

In [ ]:
a.mean(axis=0), a.std(axis=0)

### Compare Gradient Ascent with Optimal Social Welfare

In [ ]:
from comblearn.env import BruteForceOptimizer

output = []
for i in range(10):
    auction = CombinatorialAuction(cfg['auction'])
    optimal_social_welfare = auction.data_handler.opt_sw
    # _, rg_social_welfare = auction.run()
    auction.allocation_config['scheme'] = 'GradientAscent'
    auction.allocation_config['optimizer'] = 'comblearn.optim.GradientAscentOptimizer'  # Change this to batch if you want to perform a batch optimization
    auction.allocation_config['learning-rate'] = 0.001
    auction.allocation_config['batch-size'] = 100
    auction.allocation_config['eps'] = 0.00
    _, sw, _,  sw1 = auction.run()

    ws = [b.vf for b in auction.bidders]

    edsf_learned = [vf for _, vf in auction.learning_handler.models.items()]
    dsf_learned = [vf for _, vf in auction.learning_handler1.models.items()]
    
    optim_aux = BruteForceOptimizer(len(auction.items), len(auction.bidders), edsf_learned)
    opt_alloc = optim_aux.optimize()
    opt_learn_edsf_sw = social_welfare(ws, opt_alloc)

    optim_aux = BruteForceOptimizer(len(auction.items), len(auction.bidders), dsf_learned)
    opt_alloc = optim_aux.optimize()
    opt_learn_dsf_sw = social_welfare(ws, opt_alloc)

    output.append((sw, sw1, opt_learn_edsf_sw, opt_learn_dsf_sw, optimal_social_welfare))

    print('A:', (sw, sw1, opt_learn_edsf_sw, opt_learn_dsf_sw, optimal_social_welfare))

In [21]:
os = [[a.item(), b.item(), c.item(), d.item(), e.item()] for a, b, c, d, e in output]

In [ ]:
a = np.array(os)
np.mean(a, axis=0), np.std(a, axis=0)